In [1]:
import os

In [2]:
%pwd

'c:\\Users\\NARINDER\\Desktop\\new project\\research'

In [3]:
os.chdir("..")

In [4]:
%pwd

'c:\\Users\\NARINDER\\Desktop\\new project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class dataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.floodClassifier.utils.common import read_yaml, create_directories
from src.floodClassifier.constants import *

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifact_root])

    def get_data_ingestion_config(self) -> dataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = dataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [7]:
import zipfile
import os
import gdown
from src.floodClassifier import logger

class DataIngestion:
    def __init__(self, config: dataIngestionConfig):
        self.config = config

    def download_file(self) -> Path:
        if os.path.exists(self.config.local_data_file):
            logger.info(f"File already exists at {self.config.local_data_file}. Skipping download.")
        else:
            logger.info(f"Downloading file from {self.config.source_URL} to {self.config.local_data_file}")
            gdown.download(url=self.config.source_URL, output=str(self.config.local_data_file), quiet=False)
            logger.info(f"File downloaded successfully to {self.config.local_data_file}")
        return self.config.local_data_file

    def extract_zip_file(self, zip_file_path: Path):
        logger.info(f"Extracting zip file {zip_file_path} to directory {self.config.unzip_dir}")
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        logger.info(f"Extraction completed successfully to directory {self.config.unzip_dir}")

    def initiate_data_ingestion(self):
        zip_file_path = self.download_file()
        self.extract_zip_file(zip_file_path)
        return self.config.unzip_dir

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.initiate_data_ingestion()
    
except Exception as e:
    logger.exception(e)
    raise e

[2025-11-21 02:54:03,049: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-21 02:54:03,052: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-21 02:54:03,053: INFO: common: Directory created at: artifacts]
[2025-11-21 02:54:03,054: INFO: common: Directory created at: artifacts/data_ingestion]
[2025-11-21 02:54:03,055: INFO: 500249691: Downloading file from https://drive.google.com/uc?id=1aehKgW7G0hxNAukqZCm17vDIbn2_8ZGx to artifacts\data_ingestion\data.zip]


Downloading...
From: https://drive.google.com/uc?id=1aehKgW7G0hxNAukqZCm17vDIbn2_8ZGx
To: c:\Users\NARINDER\Desktop\new project\artifacts\data_ingestion\data.zip
100%|██████████| 381k/381k [00:00<00:00, 787kB/s]

[2025-11-21 02:54:06,333: INFO: 500249691: File downloaded successfully to artifacts\data_ingestion\data.zip]
[2025-11-21 02:54:06,334: INFO: 500249691: Extracting zip file artifacts\data_ingestion\data.zip to directory artifacts\data_ingestion]
[2025-11-21 02:54:06,350: INFO: 500249691: Extraction completed successfully to directory artifacts\data_ingestion]
